In [1]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly_express as px
import seaborn as sns
import umap

/home/lippincm/miniconda3/envs/Interstellar/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/lippincm/miniconda3/envs/Interstellar/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/lippincm/miniconda3/envs/Interstellar/lib/pyth

In [2]:
nELISA_plate_430420_PBMC_path = pathlib.Path(
    "../../Data/clean/Plate2/nELISA_plate_430420_PBMC.csv"
)
manual_cluster_1_path = pathlib.Path(
    "../../Data/clean/Plate2/Manual_Treatment_Clusters_1.csv"
)

manual_cluster_2_path = pathlib.Path(
    "../../Data/clean/Plate2/Manual_Treatment_Clusters_2.csv"
)


nELISA_plate_430420_PBMC = pd.read_csv(nELISA_plate_430420_PBMC_path)
manual_clusters_1 = pd.read_csv(manual_cluster_1_path)
manual_clusters_2 = pd.read_csv(manual_cluster_2_path)

nELISA_orgingal_plate = nELISA_plate_430420_PBMC.copy()

In [3]:
# select data only columns and make floats
nELISA_data_values = nELISA_orgingal_plate.filter(like="NSU", axis=1)
nELISA_data_values = nELISA_data_values.astype("float")
nELISA_data_values.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,0.398325,-0.130139,8.797693,-0.445542,0.371358,-0.462473,1.422635,-1.915471,-0.748352,-1.583011,...,-1.065390,-0.314291,0.171604,-0.137741,0.373863,-1.042447,-1.016304,-0.289143,0.373108,-0.804121
1,-0.835343,-0.174790,7.975378,-0.747735,-2.742042,-0.735577,-2.106887,-0.153780,-0.750152,-0.241556,...,-0.766995,0.190291,-1.499028,-0.660653,-0.594824,-0.400520,-0.100588,-1.055132,-0.020319,-0.946662
2,-0.933929,-1.258131,7.642998,-0.720992,-4.155766,1.830996,1.389517,-0.785248,-0.262083,-1.058364,...,0.547156,-0.793964,0.592818,-1.207418,-0.318222,0.187477,-1.584273,-0.015301,-1.393639,-2.013903
3,-0.965939,0.443343,4.022308,-1.913187,1.560739,-1.219594,0.139052,0.110671,0.179028,-0.604800,...,-1.639177,0.494727,1.050084,-2.030229,0.663452,-0.645759,1.403077,-1.946008,-1.806517,-0.911292
4,-0.961577,-0.006246,11.143199,-0.869969,-0.589156,0.173679,1.374521,-1.267405,0.044353,-0.098963,...,-0.899452,-0.197758,-0.858524,0.211425,-0.734540,-0.938241,-0.976583,0.352800,0.472133,-1.347747


In [4]:
# normalize data via max value in each column (recommendation by Nomic)
max_values = nELISA_data_values.max()  # find max value in each column
nELISA_data_values_sensor_max_norm = nELISA_data_values.div(
    max_values
)  # divide each value in each column by max value in that column
nELISA_data_values_sensor_max_norm.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,0.011740,-0.060304,0.261959,-0.152960,0.045788,-0.180913,0.389698,-0.308428,-0.282731,-0.249033,...,-0.660487,-0.133548,0.058089,-0.052617,0.128384,-0.461470,-0.224275,-0.035787,0.121201,-0.197622
1,-0.024620,-0.080994,0.237474,-0.256706,-0.338092,-0.287747,-0.577134,-0.024762,-0.283411,-0.038001,...,-0.475498,0.080858,-0.507426,-0.252369,-0.204261,-0.177302,-0.022197,-0.130593,-0.006601,-0.232653
2,-0.027526,-0.582990,0.227577,-0.247525,-0.512404,0.716260,0.380627,-0.126440,-0.099016,-0.166498,...,0.339208,-0.337370,0.200671,-0.461233,-0.109277,0.082992,-0.349612,-0.001894,-0.452713,-0.494939
3,-0.028469,0.205435,0.119768,-0.656820,0.192438,-0.477088,0.038090,0.017820,0.067637,-0.095145,...,-1.016205,0.210218,0.355457,-0.775546,0.227827,-0.285864,0.309626,-0.240855,-0.586833,-0.223960
4,-0.028341,-0.002894,0.331799,-0.298671,-0.072643,0.067941,0.376519,-0.204077,0.016757,-0.015568,...,-0.557614,-0.084031,-0.290613,0.080764,-0.252239,-0.415340,-0.215509,0.043666,0.153369,-0.331224


In [5]:
# print mean and stdev of first data column before and after normalization to check normalization
print(f"NSU nELISA mean of Activin A: {nELISA_data_values['Activin A [NSU]'].mean()}")
print(f"NSU nELISA STDEV of Activin A: {nELISA_data_values['Activin A [NSU]'].std()}")

print(
    f"NSU sensor normalized nELISA mean of Activin A: {nELISA_data_values_sensor_max_norm['Activin A [NSU]'].mean()}"
)
print(
    f"NSU sensor normalized nELISA STDEV of Activin A: {nELISA_data_values_sensor_max_norm['Activin A [NSU]'].std()}"
)

NSU nELISA mean of Activin A: 4.584872167692435
NSU nELISA STDEV of Activin A: 10.115791239471815
NSU sensor normalized nELISA mean of Activin A: 0.13513060896303342
NSU sensor normalized nELISA STDEV of Activin A: 0.2981441969015097


In [6]:
# rename columns to remove special character "/"
nELISA_orgingal_plate.columns = nELISA_orgingal_plate.columns.str.replace("/", "_")

# set umap parameters
umap_params = umap.UMAP(
    n_neighbors=6,
    min_dist=0.8,
    n_components=2,
    metric="cosine",
    spread=1.1,
    init="random",
    random_state=0,
)

# fit and transform data for umap
proj_2d = umap_params.fit_transform(nELISA_data_values_sensor_max_norm)

# add umap coordinates to dataframe of metadata and raw data
nELISA_orgingal_plate["umap_1"] = proj_2d[:, 0]
nELISA_orgingal_plate["umap_2"] = proj_2d[:, 1]

# # add manual clusters columns to dataframe
# nELISA_plate_430420 = pd.merge(
#     nELISA_orgingal_plate, manual_clusters, on=("inducer1", "inhibitor"), how="inner"
# )

# add manual clusters columns to dataframe
nELISA_plate_430420 = pd.merge(
    nELISA_orgingal_plate,
    manual_clusters_2,
    on=(
        "inducer1",
        "inducer1_concentration_value",
        "inhibitor",
        "inhibitor_concentration_value",
        "inducer2",
        "inducer2_concentration_value",
    ),
    how="inner",
)

nELISA_plate_430420["inducer1_plus_concentration"] = (
    nELISA_plate_430420["inducer1"]
    + "_"
    + nELISA_plate_430420["inducer1_concentration"]
)

nELISA_plate_430420["inducer1_plus_inhibitor"] = (
    nELISA_plate_430420["inducer1"] + "_" + nELISA_plate_430420["inhibitor"]
)

nELISA_plate_430420["inducer1_plus_concentration_plus_inhibitor"] = (
    nELISA_plate_430420["inducer1"]
    + "_"
    + nELISA_plate_430420["inducer1_concentration"]
    + "_"
    + nELISA_plate_430420["inhibitor"]
)


# define output paths
nELISA_plate_430420_out_path = pathlib.Path(
    "./results/nELISA_plate_430420_umap_PBMC.csv"
)
# write to csv
nELISA_plate_430420.to_csv(nELISA_plate_430420_out_path, index=False)